In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import sys
sys.path.append('/home/zsteineh/cnn_hilbert/cnn_hilbert_workspace')
import hilbert_DL_utils
from hilbert_DL_utils import load_data

In [10]:
model_dir = '/data1/users/gsquist/state_decoder/accuracy_outputs/cb46fd46/class_ssl/'
model_name = 'cnn_state_model_eegnet_hilb_fold_0.h5'
model_fname = model_dir + model_name

norm_rate = 0.25
wrist_lp = '/data1/users/stepeter/cnn_hilbert/ecog_data/xarray/'
pats_ids_in = ['cb46fd46']
test_day = 'last'
n_chans_all=64
tlim=[-1,1]
n_folds = 1

optimizer='adam'
loss='categorical_crossentropy'
patience = 15
early_stop_monitor='val_loss'
epochs=64
chckpt_path = '/home/zsteineh/ez_ssl_results/'

In [11]:
X,y,x_test,y_test,sbj_order_all,sbj_order_test_last = load_data(pats_ids_in, wrist_lp,
                                                              n_chans_all=n_chans_all,
                                                              test_day=test_day, tlim=tlim)

100%|██████████| 1/1 [00:00<00:00,  1.62it/s]

Data loaded!


In [12]:
nb_classes = len(np.unique(y))
order_inds = np.arange(len(y))
np.random.shuffle(order_inds)
X = X[order_inds,...]
y = y[order_inds]
order_inds_test = np.arange(len(y_test))
np.random.shuffle(order_inds_test)
# X_test = X_test[order_inds_test,...]
# y_test = y_test[order_inds_test]
y2 = np_utils.to_categorical(y-1)
y_test2 = np_utils.to_categorical(y_test-1)
X2 = np.expand_dims(X,1)
X_test2 = np.expand_dims(x_test,1)

split_len = int(X2.shape[0]*0.2)
last_epochs = np.zeros([n_folds,2])

val_inds = np.arange(0,split_len)+(0*split_len)
train_inds = np.setdiff1d(np.arange(X2.shape[0]),val_inds) #take all events not in val set

x_train = X2[train_inds,...]
y_train = y2[train_inds]
x_val = X2[val_inds,...]
y_val = y2[val_inds]

In [5]:
pretask_model = tf.keras.models.load_model(model_fname)
pretask_model.summary()

/home/zsteineh/anaconda3/envs/ez_ss_dl_venv/lib/python3.8/site-packages/tensorflow/python/keras/layers/core.py:1028: UserWarning: recurrent_NN_models is not loaded, but a Lambda layer uses it. It may cause errors.
  warnings.warn('{} is not loaded, but a Lambda layer uses it. '


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 64, 501)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 64, 501)   512         input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 64, 501)   0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 8, 64, 501)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [6]:
x = pretask_model.layers[-3].output
# x = Flatten(name = 'flatten2')(x)
x = Dense(nb_classes, name = 'dense', kernel_constraint = max_norm(norm_rate))(x)
softmax = Activation('softmax', name = 'softmax')(x)

transfer_model = Model(inputs=pretask_model.input, outputs=softmax)

# Set only last 3 layers to be trainable
for l in transfer_model.layers:
    l.trainable = False
for l in transfer_model.layers[-3:]:
    l.trainable = True #train last 3 layers

transfer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 64, 501)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 64, 501)   512         input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 64, 501)   0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 8, 64, 501)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
transfer_model.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])
checkpointer = ModelCheckpoint(filepath=chckpt_path,verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor=early_stop_monitor, mode='min',
                               patience=patience, verbose=0)

In [14]:
h = transfer_model.fit(x_train, y_train, batch_size = 16, epochs = epochs, 
                        verbose = 2, validation_data=(x_val, y_val),
                        callbacks=[checkpointer,early_stop])

Epoch 1/64


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [13]:
x_train.shape

(772, 1, 64, 501)

In [ ]:
acc_lst = []
preds = transfer_model.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = transfer_model.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = transfer_model.predict(x_test).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test.argmax(axis=-1)))

print(np.asarray(acc_lst))